## Notebook-2: OOPs Based experiment on PyTorch Dataset from Huggingface Dataset

### Experiment Contents and Objective on Notebook-1 and Notebook-2:

#### Notebook-1: Custom Function Based experiment
This notebook will cover Important part to prepare Donut Dataset for all task but here only we experimented on DOCVQA datset

- 1. Detail experiment of Donut ground truth and gt_parse structure formation and create add_ground_truth custom function
- 2. create add_tokens custom function and its embedding
- 3. json2token experiment and create custom function to make proper prmpt sequence
- 4. tokenize and PyTorch tensor conversion of input image pizel, decoder input ids and labels which is the masking clone of decodder input ids from huggingface dataset
- 5.Convert huggingface dataset to PyTorch Dataset also taking only three variables (input image pixel values, decoder input ids and labels mask tensor) which will be mainly required for our DONUT model and then these tensor will be processed for model building

#### Notebook-2: OOPs Based experiment
This notebook will be mainly focused on creating Donut Dataset class from PyTorch and it will combine all the above task inside one class using OOPs
- 1. Creating custom function for ground truth task i.e. 'gt_parses'
- 2. Creating DonutDataset class which consists of all rest task such as add_tokens, json2tokens, tokenize and tensor formation which will directly convert huggingface dataset to Pytorch dataset and will return input image pixel values, decoder input ids and labels mask tensors for further Donut model buiding task using PyTorch

### Importing necessary Dependencies

In [1]:
!pip install -q transformers
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


In [2]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import os
import json
from google.colab import data_table
import datasets
from datasets import Dataset
import json
import re
import random
from typing import Any, List, Tuple

from PIL import Image
import torch
print("CUDA available:", torch.cuda.is_available())
# from donut import DonutModel
import transformers
from transformers import DonutProcessor, VisionEncoderDecoderConfig, VisionEncoderDecoderModel, BartConfig
from torch.utils.data import DataLoader


CUDA available: False


In [5]:
model_checkpoint = "naver-clova-ix/donut-base"
max_length = 128
image_size = [1280, 960]


config = VisionEncoderDecoderConfig.from_pretrained(model_checkpoint)
config.encoder.image_size = image_size # (height, width)
# update max_length of the decoder (for generation)
config.decoder.max_length = max_length

processor = DonutProcessor.from_pretrained(model_checkpoint)
model = VisionEncoderDecoderModel.from_pretrained(model_checkpoint, config=config)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


## S-1: Intial Steps till gt_parse or Groud Truth

In [6]:
train_path_main = '/content/drive/MyDrive/docVqa_dataset/train/train_v1.0.json'
# Open the JSON file
with open(train_path_main, 'r') as file:
  # Load the contents of the file into a dictionary
  data_train = json.load(file)

## Convert 'data' list into dataframe by passing into the dataframe
df_train = pd.DataFrame(data_train['data'])

## Making subset of the training dataframe
df_sub = df_train.iloc[:10]

base_train_path = '/content/drive/MyDrive/docVqa_dataset/train'

# Define a function to join the base path with the image path
def join_paths(image_path):
    return os.path.join(base_train_path, image_path)

# Apply the function to create a new column with the full image path
df_sub['full_path_image'] = df_sub['image'].apply(join_paths)


def extract_ocr_info(df, base_train_dir):

  dir_list = os.listdir(base_train_dir)
  ocr_base_dir = os.path.join(base_train_dir, dir_list[1])

  # Create empty lists for bounding boxes and word lists
  bbox_list = []
  word_list = []

  # Iterate over each row in the DataFrame
  for _, row in df.iterrows():

    # Construct the ocr_path for the row
    ocr_path = os.path.join(ocr_base_dir, row['ucsf_document_id'] + '_' + row['ucsf_document_page_no'] + '.json')

    # Open the json file and load the data
    with open(ocr_path, 'r') as f:
      ocr_json = json.load(f)

    # Extract the bounding boxes and word lists from the json data
    bbox_row = []
    word_row = []
    for i in ocr_json['recognitionResults'][0]['lines']:
      for j in i['words']:
        bbox_row.append(j['boundingBox'])
        word_row.append(j['text'])

    # Append the bbox and word lists to the global lists
    bbox_list.append(bbox_row)
    word_list.append(word_row)
  # Create new columns in the DataFrame to store the bbox and words lists
  df['bounding_boxes'] = bbox_list
  df['word_list'] = word_list

  df.drop(['questionId', 'image','ucsf_document_id', 'ucsf_document_page_no'], axis=1, inplace=True)

  # Return the updated DataFrame
  return df


df_sub_ocr = extract_ocr_info(df_sub, base_train_path)

## Convert dataframe into dataset object
hf_sub_ocr = Dataset.from_pandas(df_sub_ocr)


<ipython-input-6-f4858d3802c0>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['full_path_image'] = df_sub['image'].apply(join_paths)
<ipython-input-6-f4858d3802c0>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bounding_boxes'] = bbox_list
<ipython-input-6-f4858d3802c0>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [7]:
hf_sub_ocr

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list'],
    num_rows: 10
})

In [8]:
hf_sub_ocr.column_names

['question',
 'docId',
 'answers',
 'data_split',
 'full_path_image',
 'bounding_boxes',
 'word_list']

In [9]:
hf_sub_ocr.features

{'question': Value(dtype='string', id=None),
 'docId': Value(dtype='int64', id=None),
 'answers': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'data_split': Value(dtype='string', id=None),
 'full_path_image': Value(dtype='string', id=None),
 'bounding_boxes': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'word_list': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [10]:
len(hf_sub_ocr)

10

## 1. ground_truth experiment & gt_parse prompt preparation
- custom function creation for gt_parse for all kind of task

In [11]:
hf_sub_ocr['question'][0]

'what is the date mentioned in this letter?'

In [12]:
for ques in hf_sub_ocr['question']:
  print(ques, "|||||",type(ques))

what is the date mentioned in this letter? ||||| <class 'str'>
what is the contact person name mentioned in letter? ||||| <class 'str'>
Which corporation's letterhead is this? ||||| <class 'str'>
Who is in  cc in this letter? ||||| <class 'str'>
what is the subject of  this letter? ||||| <class 'str'>
What is the number at the bottom of the page, in bold? ||||| <class 'str'>
How many points are there in modifications to readout instrumentation ||||| <class 'str'>
what is the date in the letter ||||| <class 'str'>
To whom this is addressed ||||| <class 'str'>
Who sent the letter? ||||| <class 'str'>


## .replace('"', '\\"')
Certainly! Let's consider an example to understand the code `questions = questions.replace('"', '\\"')`:

```python
questions = 'What is your "name"?"'
questions = questions.replace('"', '\\"')
print(questions)
```

In this example, we have a string `questions` that contains a question: "What is your "name"?" We want to replace the double quote character (`"`) within the string with a backslash followed by a double quote (`\"`).

The line `questions = questions.replace('"', '\\"')` performs the replacement. It searches for all occurrences of the double quote (`"`) character in the `questions` string and replaces them with `\"`. The resulting modified string is assigned back to the variable `questions`.

When we print the value of `questions`, the output will be:
```
What is your \"name\"?
```

The original double quotes within the string are replaced with `\"`, where the backslash `\` serves as an escape character to indicate that the following double quote should be treated as part of the string, rather than as a closing delimiter.

So, the purpose of the code is to modify the original string by escaping double quotes with a backslash, allowing them to be properly interpreted within the string.

In [13]:
hf_sub_ocr['answers'][0]

['1/8/93']

In [14]:
for ans in hf_sub_ocr['answers']:
  print(ans, "|||||",type(ans))

['1/8/93'] ||||| <class 'list'>
['P. Carter', 'p. carter'] ||||| <class 'list'>
['Brown & Williamson Tobacco Corporation'] ||||| <class 'list'>
['T.F. Riehl'] ||||| <class 'list'>
['Review of existing Brainstorming Ideas/483'] ||||| <class 'list'>
['499150498'] ||||| <class 'list'>
['5.', '5'] ||||| <class 'list'>
['June 11, 1990', 'June 11,1990'] ||||| <class 'list'>
['Dr.K.S.Houghton'] ||||| <class 'list'>
['Ted Sanders'] ||||| <class 'list'>


In [15]:
ans1_before = hf_sub_ocr['answers'][2]
ans1_before

['Brown & Williamson Tobacco Corporation']

In [16]:
ans1_after = ans1_before[0] .replace('"', '\"')
ans1_after

'Brown & Williamson Tobacco Corporation'

### Note structure of the ground truth for different Task:
```
GT for QA Task:

## For Single Answer
'{"gt_parses": [{"question" : "...........", "answer" : "........"}]}'

or
## For Multiple Answer
'{"gt_parses": [{"question" : "...........", "answer1" : "........",
                                    "answer2" : "........", "answer3" : "........"..so on}]}'

```

```
GT for Doc Classification Task:

'{"gt_parse": {"class" : "class_name"}}'

```

```
Sample helping code:

template = '{"gt_parse": {"class" : '

def add_ground_truth(hf_examples):

  ground_truths = []

  for label in hf_examples['label']:
    ground_truths.append(template + '"' + id2label[label] + '"' + "}}")

  hf_examples['ground_truth'] = ground_truths

  return hf_examples

## Mapping with the
hf_dataset_gt = hf_dataset.map(add_ground_truth, batched=True)

```


```
GT for Doc Parsing task:
{"gt_parses": [{"company": "ADVANCO COMPANY",
"date": "17/01/2018",
"address": "NO 1&3, JALAN WANGSA DELIMA 12, WANGSA LINK, WANGSA MAJU, 53300 KUALA LUMPUR",
"total": "7.00"}]}

```
```
Sample helping code:

parse = {
  "company": "ADVANCO COMPANY",
  "date": "17/01/2018",
  "address": "NO 1&3, JALAN WANGSA DELIMA 12, WANGSA LINK, WANGSA MAJU, 53300 KUALA LUMPUR",
  "total": "7.00"
}

print(parse, type(parse))

gt_example = '{"gt_parses": [{'
# print(gt_example)

for key, value in parse.items():
  value = value.replace('"', '\"')
  gt_example += '"' + key + '": "' + value + '", '
  # print(gt_example)
gt_example = gt_example.rstrip(', ')
gt_example += '}]}'
print(gt_example)

```

```
Sample Custom Functom:
def add_ground_truth(hf_examples):

  keys = hf_examples.keys()  # Get the keys of the example dictionary

  ground_truths = []    # Creating an empty list for ground truth prompts

  for example in hf_examples:
    ground_truth_example = '{"gt_parses": [{'  # Start of the ground truth prompt string

    # Iterate over the keys and values of each example
    for key, value in example.items():
      value = value.replace('"', '\"')   # Replacement of " from each value
      ground_truth_example += '"' + key + '": "' + value + '", '  # Append key-value pairs to the ground truth prompt

    ground_truth_example = ground_truth_example.rstrip(', ')  # Remove the trailing comma and space
    ground_truth_example += ']}]'  # End of the ground truth prompt string

    ground_truths.append(ground_truth_example)  # Add the ground truth prompt to the list

  hf_examples['ground_truth'] = ground_truths  # Add the ground truth prompts to the example dictionary

  return hf_examples


hf_dataset_gt = hf_dataset.map(add_ground_truth, batched=True)

print(hf_dataset_gt['ground_truth'][0])
```

In [17]:
# Creating custom function for DOCVQA task

def add_ground_truth(hf_examples):
  images = hf_examples['full_path_image']
  questions = hf_examples['question']
  answers = hf_examples['answers']

  ground_truths = []    # Creating empty list
  for image,question, answer in zip(images,questions, answers):
    # we need to escape " characters appearing in the query and/or answer
    question = question.replace("\\", "") ## this was just one corrupt example (index 91 of training set)
    question = re.sub(' +', ' ', question)
    question = question.replace('"', '\\"')   # replacement of " from question
    # let's create the ground truth string
    ground_truth_example = '{"gt_parses": ['
    for idx, answ in enumerate(answer):
      # ans = answ.replace('"', '\"')   # replacement of " from answer
      answ = answ.replace("\\", "")
      ans = answ.replace('"', '\\"')
      ground_truth_example += '{"question" : "' + question + '", "answer" : "' + ans + '"}'

      # add comma for more than one element present in the answer list
      if idx != len(answer) - 1:   # when current index is not equal to last index
        ground_truth_example += ', '
    ground_truth_example += ']}'
    ground_truths.append(ground_truth_example)    ## appending ground_truths list for every row

  hf_examples['ground_truth'] = ground_truths

  return hf_examples

In [18]:
hf_sub_ocr_gt = hf_sub_ocr.map(add_ground_truth, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

## Note: Why 'batched = True' parameter?
- When 'batched = True', it suggests that the code or function is designed to handle data in batches. The exact behavior and implications of this parameter can vary depending on the specific code or function being used.

- Typically, when processing data in batches, it means that instead of processing individual examples one by one, the code or function operates on a group of examples simultaneously. This can improve computational efficiency by leveraging parallel processing and optimizing memory usage.

In [19]:
hf_sub_ocr_gt

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list', 'ground_truth'],
    num_rows: 10
})

In [20]:
print(hf_sub_ocr_gt['ground_truth'][2], '|||||' , type(hf_sub_ocr_gt['ground_truth'][2]))

{"gt_parses": [{"question" : "Which corporation's letterhead is this?", "answer" : "Brown & Williamson Tobacco Corporation"}]} ||||| <class 'str'>


In [21]:
print(hf_sub_ocr_gt['ground_truth'][1], '|||||' , type(hf_sub_ocr_gt['ground_truth'][1]))

{"gt_parses": [{"question" : "what is the contact person name mentioned in letter?", "answer" : "P. Carter"}, {"question" : "what is the contact person name mentioned in letter?", "answer" : "p. carter"}]} ||||| <class 'str'>


## **S-2: DonutDataset Class creation by combining all other custom function for all type of task

In [22]:
from torch.utils.data import Dataset

In [23]:
## Final Class by combining altogether

new_special_tokens_list = []
# list with the newly added tokens. The purpose of this list is to keep track of the tokens that were added to the tokenizer.

## DonutDataset class inherits from the torch.utils.data.Dataset class and overrides its methods

class DonutDataset(Dataset):
  """
  Args:
      dataset_name_or_path: name of dataset (available at huggingface.co/datasets) or the path containing image files and metadata.jsonl
      max_length: the max number of tokens for the target sequences
      split: whether to load "train", "validation" or "test" split
      ignore_id: ignore_index for torch.nn.CrossEntropyLoss
      prompt_start_token: the special token at the start of the sequences
      prompt_end_token: the special token at the end of the sequences
      sort_json_key: whether or not to sort the JSON keys
  """

  def __init__(
    self,
    dataset_name_or_path: str,    # Name of the dataset or the path containing huggingface datset
    max_length: int,              # the maximum number of tokens for the target sequences/prompt
    # split: str = "train",
    ignore_id: int = -100,        # ignore_index for torch.nn.CrossEntropyLoss
    prompt_start_token: str = None,   # The special token at the start of the propmpt sequences
    prompt_end_token: str = None,   # The special token at the end of the prompt sequences
    sort_json_key: bool = True,       # Whether or not to sort the JSON keys
):
    super().__init__()

    ## Variable Declaration

    self.max_length = max_length
    # self.split = split
    self.ignore_id = ignore_id
    self.prompt_start_token = prompt_start_token
    self.prompt_end_token = prompt_end_token
    self.sort_json_key = sort_json_key

    self.dataset = dataset_name_or_path
    self.dataset_length = len(self.dataset)   # The length of the dataset is stored

    self.gt_prompt_sequences = []    # operation for every single row  *** Note: same as hf_examples['gt_prompt'] which we did for single custom function case
    # list stores the ground truth (gt) token sequences, which are generated using the json2token() method for each sample in the dataset

    ## S-1: Iterate over all rows of the dataset and creating gt_prompt from gt_parses

    for sample in self.dataset:   # each sample is a dictionary

      # print(sample["ground_truth"], '|||||', type(sample["ground_truth"]))

      ground_truth_jsob_obj = json.loads(sample["ground_truth"])   # convert a string representation of ground_truth a JSON object
      if "gt_parses" in ground_truth_jsob_obj:  # when multiple ground truths are available, e.g., docvqa
        assert isinstance(ground_truth_jsob_obj["gt_parses"], list)
        gt_jsons_list = ground_truth_jsob_obj["gt_parses"]    ## Stores the list of gt_parses value as gt_parses is the only key
      else:
        assert "gt_parse" in ground_truth_jsob_obj and isinstance(ground_truth_jsob_obj["gt_parse"], dict)
        gt_jsons_list = [ground_truth_jsob_obj["gt_parse"]]


      ## Appending the prompt using list comprehension with adding cls_token before and eos_token at the end
      # For DOCVQA task
      self.gt_prompt_sequences.append(
            [
              processor.tokenizer.cls_token +
              self.json2token(
                  gt_jsons_list,
                  # update_special_tokens_for_json_key=self.split == "train",
                  sort_json_key=self.sort_json_key,
                )
              + processor.tokenizer.eos_token

              # for gt_json in gt_jsons_list  # iterate over every dictionary json from the gt_jsons list
            ]
        )
      # #  Doc Classification & Parsing task
      # self.gt_token_sequences.append(
      #     [
      #       processor.tokenizer.cls_token +
      #       self.prompt_start_token +
      #       self.json2token(
      #           gt_json,
      #           # update_special_tokens_for_json_key=self.split == "train",
      #           sort_json_key=self.sort_json_key,
      #         )
      #       + self.prompt_end_token
      #         + processor.tokenizer.eos_token

      #         for gt_json in gt_jsons_list  # iterate over every dictionary json from the gt_jsons list
      #     ]
      # )


    self.add_tokens([self.prompt_start_token, self.prompt_end_token])
    self.prompt_start_token_id = processor.tokenizer.convert_tokens_to_ids(self.prompt_start_token)
    self.prompt_end_token_id = processor.tokenizer.convert_tokens_to_ids(self.prompt_end_token)

    # print(self.gt_prompt_sequences)

  ## S-2: Function converts an ordered JSON object into a token sequence
  def json2token(self, json_obj: Any,  sort_json_key: bool = True):
    """
    Convert an ordered JSON object into a token sequence
    """
    if type(json_obj) == dict:
      if len(json_obj) == 1 and "text_sequence" in json_obj:
        return json_obj["text_sequence"]
      else:
        output = ""
        if sort_json_key:
          keys = sorted(json_obj.keys(), reverse=True)
        else:
          keys = json_obj.keys()

        for k in keys:
          # if update_special_tokens_for_json_key:
            # self.add_tokens([fr"", fr""])
          self.add_tokens([fr"<s_{k}>", fr"</s_{k}>"])
          output += (
              fr"<s_{k}>"
              + self.json2token(json_obj[k], sort_json_key)
              + fr"</s_{k}>"
          )
        return output

    elif type(json_obj) == list:
      return r"<sep/>".join([self.json2token(item, sort_json_key) for item in json_obj])

    else:
      json_obj = str(json_obj)
      if f"<{json_obj}/>" in new_special_tokens_list:
        json_obj = f"<{json_obj}/>"  # for categorical special tokens
      return json_obj

  ## S-3: Custom function to add new starting and ending prompt tokens and embed it
  def add_tokens(self, list_of_tokens: List[str]):
    """
    Add special tokens to tokenizer and resize the token embeddings of the decoder
    """
    newly_added_num = processor.tokenizer.add_tokens(list_of_tokens)
    if newly_added_num > 0:
      model.decoder.resize_token_embeddings(len(processor.tokenizer))
      new_special_tokens_list.extend(list_of_tokens)

  ## S-4: Aligns the dataset length with the valid indices
  def __len__(self) -> int:
    return self.dataset_length - 1

  ## S-5: special method that allows objects to support indexing and accessing their elements using square bracket notation ([]).
  # It is invoked when you use square brackets to access an element of an object, like obj[key]
  def __getitem__(self, indx: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Load image from image_path of given dataset_path and convert into input pixel value tensor
    Convert ground truth data into input_ids (tokenized string) which is nothing but decoder input ids
    Create clone tensor of decoder input ids for masking and model understanding task
    Returns:
        input_tensor : preprocessed image
        input_ids : tokenized gt_data
        labels : masked labels (model doesn't need to predict prompt and pad token)
    """
    sample = self.dataset[indx]

    # S-5.1: create input_tensor from image i.e pixel values
    try:
      ## ** Note: This image path part will vary as per task
      # pixel_values = processor(Image.open(sample['full_path_image']).convert("RGB"), random_padding=self.split == "train", return_tensors="pt").pixel_values
      pixel_values = processor(Image.open(sample['full_path_image']).convert("RGB"), return_tensors="pt").pixel_values
      # torch.Size([1, 3, 2560, 1920]) shape before squeeze
      input_tensor_pixel = pixel_values.squeeze()

    except Exception as e:
      print(sample)
      print(f"Error: {e}")
      return {}

    # S-5.2: tokenize document and creating decoder input ids

    processed_prompt = self.gt_prompt_sequences[indx]  # can be more than one, e.g., DocVQA Task 1
    # random.choice() is in scenarios where you need to select a random element from a collection of items.
    decoder_input_ids = processor.tokenizer(
        self.gt_prompt_sequences[indx],
        add_special_tokens=False,
        max_length=self.max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].squeeze(0)   #torch.Size([1, 128]) before squeeze

    # S-5.3: Create labels which is clone tensor for extracted values from decoder input ids tensors

    labels = decoder_input_ids.clone()    ## clone tensor of decoder input ids

    prompt_start_token_id_index = (labels == self.prompt_start_token_id).nonzero
    prompt_end_token_id_index = (labels == self.prompt_end_token_id).nonzero

    ## Creating empty integer unmask tensor
    unmask = torch.tensor([], dtype=torch.int)

    ## iterate over both start and end tensor simulteneously
    for start_index,end_index in zip(prompt_start_token_id_index(), prompt_end_token_id_index()):
      val_extracted = labels[start_index.squeeze()+1:end_index.squeeze()]     ## stores all extarcted values
      unmask = torch.cat((unmask, val_extracted))     ## concatenating start and end index value

    labels[~torch.isin(labels, unmask)] =  self.ignore_id       ## assign with ignore id

    return input_tensor_pixel, decoder_input_ids, labels

      # if self.split == "train":
      #     labels = input_ids.clone()
      #     labels[
      #         labels == processor.tokenizer.pad_token_id
      #     ] = self.ignore_id  # model doesn't need to predict pad token
      #     labels[
      #         : torch.nonzero(labels == self.prompt_end_token_id).sum() + 1
      #     ] = self.ignore_id  # model doesn't need to predict prompt (for VQA) e.g. here "<s_answer>"
      #     return input_tensor, input_ids, labels
        ##  Here input_tensor = pixel value, input_ids = decoders prompt ids, labels = as per choice and remove the portion by -100 to make model understand
      # else:
      #     prompt_end_index = torch.nonzero(
      #         input_ids == self.prompt_end_token_id
      #     ).sum()  # return prompt end index instead of target output labels
      #     return input_tensor, input_ids, prompt_end_index, processed_parse


In [24]:
processor.image_processor.size = image_size[::-1] # should be (width, height)
processor.image_processor.do_align_long_axis = False

train_dataset_sub = DonutDataset(hf_sub_ocr_gt, max_length=max_length,
                            prompt_start_token="<s_answer>",
                            prompt_end_token="</s_answer>",
                            sort_json_key=False,
                             )

## For Doc Classification task
# prompt_start_token = "<s_docClass>"
# prompt_end_token = "</s_docClass>"

## For Doc Prsing task
# prompt_start_token = "<s_docParse>"
# prompt_end_token = "</s_docParse>"

In [25]:
train_dataset_sub

In [26]:
len(train_dataset_sub)

9

In [27]:
train_dataset_sub[0]

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 tensor([    0, 57525, 27973, 52743, 48941, 44960, 40053, 35647, 25515, 54964,
   

In [28]:
## Sample check of row 0
pixel_values_0, decoder_input_ids_0, labels_0  = train_dataset_sub[0]
for decoder_input_id, label in zip(decoder_input_ids_0.tolist()[:-1], labels_0.tolist()[:]):
  if label != -100:
    print(processor.decode([decoder_input_id]), processor.decode([label]))
  else:
    print(processor.decode([decoder_input_id]), label)

<s> -100
<s_question> -100
what -100
is -100
the -100
date -100
mention -100
ed -100
in -100
this -100
letter -100
? -100
</s_question> -100
<s_answer> -100
1 1
/8 /8
/ /
93 93
</s_answer> -100
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad

In [29]:
train_dataset_sub[1]        ## details of row 1

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 tensor([    0, 57525, 27973, 52743, 48941, 44474, 37517, 55856, 40053, 35647,
   

In [30]:
## Sample check of row 1
pixel_values_1, decoder_input_ids_1, labels_1  = train_dataset_sub[1]
for decoder_input_id, label in zip(decoder_input_ids_1.tolist()[:-1], labels_1.tolist()[:]):
  if label != -100:
    print(processor.decode([decoder_input_id]), processor.decode([label]))
  else:
    print(processor.decode([decoder_input_id]), label)

<s> -100
<s_question> -100
what -100
is -100
the -100
contact -100
person -100
name -100
mention -100
ed -100
in -100
letter -100
? -100
</s_question> -100
<s_answer> -100
P P
. .
Car Car
ter ter
</s_answer> -100
<sep/> -100
<s_question> -100
what -100
is -100
the -100
contact -100
person -100
name -100
mention -100
ed -100
in -100
letter -100
? -100
</s_question> -100
<s_answer> -100
p p
. .
carte carte
r r
</s_answer> -100
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
